# 1 Author

**Student Name**:  Shivam Tyagi

**Student ID**:  220428206




# 2 Problem formulation

Building a Machine Learning model which aims at predictng whether an audio is recorded indoors or outdoors.To clearly predict the location type whether indoors or outdoors, various features that an audio data posesses are explored and the models efficiency is tested using the acuuracy score along with confusion matrix.

It is interesting because by using only few locations we can train the classifier so that it can predict whether or not any recording is outdoor or indoor, irrespective of the location in future.

# 3 Machine Learning pipeline

1. **Data Load**: Loading the data files in .wav format from personal google drive.

2. **Label Conversion**:  Converted the labels as 0(outdoors) and 1(indoors). 

3. **Feature Extraction**: Specific spectral features related to audio files such as: centroid, bandwidth, mfcc etc were extracted.

4. **Component Analysis**: Following feature extraction, Principal component analysis was performed to take out only the most contributing features(8 in our case).

5. **Data split**: The whole dataset is then split into training and validation with 70:30 raito.

6. **Standardisation**: The training and validation data is then normalized using mean and standard deviation, X = (X-mean)/sd.

7. **ML Model**: Different ML models like SVC, RandomForest are trained and the best is selected according to point number 8, validation scores. 

8. **Validations**: Perfromance of the model is then judged on the basis of accuracy,precision, recall,f1 score and confusion matrix. All of the above are calculated for the training as well as validation datset. 


**Note**

There are few cells which does not require to be run as these are:
1. Connected to my personal google drive and hence there would be problem while granting permission.
2. It will take a lot of time to run all the 2500 file again.

Hence I have run the cells, extracted the required features and then converted them to csv instead of running and loading the cells again and again and waiting for them to try anything new.

Thus requesting to use the csv of the features and labels that i am providing along with the notebook to run and check the code.

I would write a coment at top of each cell which is not required to be run.

In [8]:
from google.colab import drive

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os, sys, re, pickle, glob
import urllib.request
import zipfile

import IPython.display as ipd
from tqdm import tqdm
import librosa
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier  




In [9]:
# Run only to verify
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Run only to verify
directory_to_extract_to1 = '/content/drive/MyDrive/Data/MiniProject/sample1/'
directory_to_extract_to2 = '/content/drive/MyDrive/Data/MiniProject/sample2/'
directory_to_extract_to3 = '/content/drive/MyDrive/Data/MiniProject/sample3/'
directory_to_extract_to4 = '/content/drive/MyDrive/Data/MiniProject/sample4/'
directory_to_extract_to5 = '/content/drive/MyDrive/Data/MiniProject/sample5/'

zip_path=[]
zip_path1 = '/content/drive/MyDrive/Data/MiniProject/MLEndLS_1.zip'
zip_path2 = '/content/drive/MyDrive/Data/MiniProject/MLEndLS_2.zip'
zip_path3 = '/content/drive/MyDrive/Data/MiniProject/MLEndLS_3.zip'
zip_path4 = '/content/drive/MyDrive/Data/MiniProject/MLEndLS_4.zip'
zip_path5 = '/content/drive/MyDrive/Data/MiniProject/MLEndLS_5.zip'


with zipfile.ZipFile(zip_path1, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to1)

with zipfile.ZipFile(zip_path2, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to2)

with zipfile.ZipFile(zip_path3, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to3)

with zipfile.ZipFile(zip_path4, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to4)

with zipfile.ZipFile(zip_path5, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to5)

sample_path1 = '/content/drive/MyDrive/Data/MiniProject/sample1/*.wav'
sample_path2 = '/content/drive/MyDrive/Data/MiniProject/sample2/*.wav'
sample_path3 = '/content/drive/MyDrive/Data/MiniProject/sample3/*.wav'
sample_path4 = '/content/drive/MyDrive/Data/MiniProject/sample4/*.wav'
sample_path5 = '/content/drive/MyDrive/Data/MiniProject/sample5/*.wav'
files1 = glob.glob(sample_path1)
files2 = glob.glob(sample_path2)
files3 = glob.glob(sample_path3)
files4 = glob.glob(sample_path4)
files5 = glob.glob(sample_path5)

MLENDLS_df = pd.read_csv('MLEndLS.csv').set_index('file_id') 
MLENDLS_df1=MLENDLS_df[0:497]
MLENDLS_df2=MLENDLS_df[497:997]
MLENDLS_df3=MLENDLS_df[997:1497]
MLENDLS_df4=MLENDLS_df[1497:1997]
MLENDLS_df5=MLENDLS_df[1997:2500] 


# 4 Transformation stage

Three types of transformations have been done:
1. **Output Label Conversions**:

*   As we are interested only in checking that if the audio was recorded outdoors or indoors hence we have just two classes to predict.

*   We assign the value of labels as 1 for indoors and 0 for outdoors, by doing so we have converted the labels from categorical data to numeric data and hence easy to understand for most of the ML models.

2. **Feature Extraction**:

*   Any outdoor recording will be more noisy compared to an indoor one hence spectral features like bandwidth, centroid, flatness and zero crossing rates are chosen which will have a higher value for outdoors as compared to indoors.
*   Other features such as pitch, power are also used which are more prevelant for outdoor audios.

*   MFCC is used for audio similarity measures which will give similar outputs for similar type of audios.


3. **Input Feature Conversions**:


*   As all the features exctracted are in the form of numpy array hence  taking their mean and then they are given as input lables which would then be used as inputs to the classifier.
*   Not all the features would be contributing as desired, hence Principal Component Analysis is performed on these input labels, value of the number of components is increased until the highest accuracy is acheived.

*   PCA with number of components =8 gives the highest accuracy and hence giving those as the inputs to the classifier.









In [10]:
# Run only to verify
def getPitch(x,fs,winLen=0.02):
  #winLen = 0.02 
  p = winLen*fs
  frame_length = int(2**int(p-1).bit_length())
  hop_length = frame_length//2
  f0, voiced_flag, voiced_probs = librosa.pyin(y=x, fmin=80, fmax=450, sr=fs,
                                                 frame_length=frame_length,hop_length=hop_length)
  return f0,voiced_flag

In [11]:
# Run only to verify
def getXy(files,labels_file, scale_audio=False, onlySingleDigit=False):
  X,y =[],[]
  for file in tqdm(files):
    fileID = file.split('/')[-1]
    file_name = file.split('/')[-1]
    yi = labels_file.loc[fileID]['in_out']=='indoor'

    fs = None # if None, fs would be 22050
    x, fs = librosa.load(file,sr=fs)
    mfcc = librosa.feature.mfcc(y = x, sr = fs, hop_length=513, n_mfcc=13) #for audio similarity measures
    cent = librosa.feature.spectral_centroid(y=x, sr=fs)
    spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=fs)
    S = np.abs(librosa.stft(x))
    contrast = librosa.feature.spectral_contrast(S=S, sr=fs)
    flatness = librosa.feature.spectral_flatness(y=x)
    zrcr=librosa.feature.zero_crossing_rate(x)
    
    if scale_audio: x = x/np.max(np.abs(x))
    f0, voiced_flag = getPitch(x,fs,winLen=0.02)
      
      
    power = np.sum(x**2)/len(x)
    pitch_mean = np.nanmean(f0) if np.mean(np.isnan(f0))<1 else 0
    pitch_std  = np.nanstd(f0) if np.mean(np.isnan(f0))<1 else 0
    voiced_fr = np.mean(voiced_flag)

    xi = [power,pitch_mean,pitch_std,voiced_fr,np.mean(mfcc),np.mean(cent),np.mean(spec_bw),np.mean(contrast),np.mean(flatness),np.mean(zrcr)]
    X.append(xi)
    y.append(yi)

  return np.array(X),np.array(y)

In [ ]:
# Run only to verify
X1,y1 = getXy(files1, labels_file=MLENDLS_df, scale_audio=True, onlySingleDigit=True)
X2,y2 = getXy(files2, labels_file=MLENDLS_df2, scale_audio=True, onlySingleDigit=True)
X3,y3 = getXy(files3, labels_file=MLENDLS_df3, scale_audio=True, onlySingleDigit=True)
X4,y4 = getXy(files4, labels_file=MLENDLS_df4, scale_audio=True, onlySingleDigit=True)
X5,y5 = getXy(files5, labels_file=MLENDLS_df5, scale_audio=True, onlySingleDigit=True)
X=np.append(X1,X2,axis=0)
X=np.append(X,X3,axis=0)
X=np.append(X,X4,axis=0)
X=np.append(X,X5,axis=0)

y=np.append(y1,y2,axis=0)
y=np.append(y,y3,axis=0)
y=np.append(y,y4,axis=0)
y=np.append(y,y5,axis=0)

ys=pd.DataFrame(y)
ys.to_csv("Label.csv")

Z=pd.DataFrame(X)
Z.to_csv("Features.csv")

In [3]:
# Start from here after importing libraries, as up till now after extracting the features the csv is converetd and i am providing the same 
Z=pd.read_csv("Features.csv")
Z.drop(Z.columns[0], axis=1, inplace=True)

from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
scalar.fit(Z)
scaled_data = scalar.transform(Z)

from sklearn.decomposition import PCA
pca = PCA(n_components = 8)
pca.fit(scaled_data)

x_pca = pca.transform(scaled_data)

X=np.array(x_pca)
ys=pd.read_csv("Label.csv")
ys.drop(ys.columns[0], axis=1, inplace=True)
y=np.array(ys)

print(X.shape)
print(y.shape)
y=y.ravel()

(2487, 8)
(2487, 1)


# 5 Modelling

1. **SVM**: SVM SVC classifier is trained because it is majorly used for the data which does not have a known distribution i.e., audio data in our case. Also it does not suffer overfitting and have very effect of outliers which may be some audios with huge amount of noise indoors as well in our case and hence could have caused issues in prediction. It gave the best results with cost parameter =2.

2. **Random Forest Classifier**: Random Forest Classifier was trained as it handles non-linearity of parameters effeiciently, and also like SVM it is also robust to outlier. But it is not chosen because it was overfitting with incresing n_estimators used(10,20,30) on the training dataset, without any significant change in the validation accuracy, whcih can be checked if "Rand" is given as choice in below code.

3. **KNeighbour Classifier**: It was trained as through KNN new data can be added seamlessly which will not impact the accuracy of the algorithm. The KNN model has ben used with n_neighbors=[5, 10, 50]. As 70% of the data (1700+) is used for training the data, so about 850 data rows were present for each class while training. But it was not chosen because of lower accuracy than SVM.

At last SVM was chosen because of higher accuracy on validation datset with cost parameter value as 2 and without any overfitting on the training dataset.


In [7]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.3,random_state=10)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

choice="Svc"

if choice=="Rand":
  print("RandomForest:")
  model  = RandomForestClassifier(n_estimators=10)
elif choice=="Svc":
  print("SVC:")
  model  = svm.SVC(C=2)
else:
    print("KNN")
    model=KNeighborsClassifier(n_neighbors=50) 

model.fit(X_train,y_train)



SVC:


SVC(C=2)

# 6 Methodology

The model training is already explianed in the previous section.

Performance of the modles were accessed through the following:

1. Accuracy: Accuracy is the ratio of correct predictions out of all predictions made by an algorithm. It can be calculated by dividing precision by recall or as 1 minus false negative rate (FNR) divided by false positive rate (FPR).
2. Precision: The Precision is the ratio of true positives over the sum of false positives and true negatives. It is also known as positive predictive value.
3. Recall: Recall is the ratio of correctly predicted outcomes to all predictions. It is also known as sensitivity or specificity.
4. f1 score: The F1-score combines these three metrics into one single metric that ranges from 0 to 1 and it takes into account both Precision and Recall.
5. Confusion matrix: This matrix tell us all the correctly clasified along with incorrectly classified labels in the form of a matrix hence easier to read.

All of the above were kept into account while chosing the final classifier and their results were compared. SVM got the highest score for them all and hence was chose to finally train the data.

# 7 Dataset

1. 2487 audio files have been used in total out of which 1176 are outdoors and 1311 are indoors.
2. Input features and output label conversion is done for all the files already explained earlier which is then followed by PCA to select only relevant input features.
3. The total files are then divided into 70:30 raito for training and test data split. 
4. The 70% data is then used to train the classifier and the remaining 30% used to validate the classifier.

All the code for feature extraction, label conversion and data split is already done in above sections and explained.


# 8 Results



In [5]:
yt_p = model.predict(X_train)
yv_p = model.predict(X_val)

pt=precision_score(yt_p,y_train)
rt=recall_score(yt_p,y_train)
f1t=f1_score(yt_p,y_train)
mt=metrics.confusion_matrix(yt_p,y_train)


pv=precision_score(yv_p,y_val)
rv=recall_score(yv_p,y_val)
f1v=f1_score(yv_p,y_val)
mv=metrics.confusion_matrix(yv_p,y_val)

print('Training Accuracy', np.mean(yt_p==y_train),"\nTraining precision",pt,"\nTraining recall",rt,"\nTrianing f1 score",f1t)
print("Confusion Matrix for training period:",mt,"\n")

print('Validation  Accuracy', np.mean(yv_p==y_val),"\nValidation precision",pv,"\nValidation recall",rv,"\nValidation f1 score",f1v)
print("Confusion Matrix for validation period:",mv)

Training Accuracy 0.7367816091954023 
Training precision 0.7657550535077289 
Training recall 0.7116022099447514 
Trianing f1 score 0.7376861397479955
Confusion Matrix for training period: [[638 197]
 [261 644]] 

Validation  Accuracy 0.6746987951807228 
Validation precision 0.7014925373134329 
Validation recall 0.6216931216931217 
Validation f1 score 0.6591865357643758
Confusion Matrix for validation period: [[269 100]
 [143 235]]


In [6]:
mean = np.mean(X_train,axis=0)
sd=np.std(X_train,axis=0)
mean1 = np.mean(X_val,axis=0)
sd1=np.std(X_val,axis=0)
X_train = (X_train-mean)/sd
X_val  = (X_val-mean1)/sd1

choice="Svc"
if choice=="Rand":
  print("RandomForest:")
  model  = RandomForestClassifier(n_estimators=10)
elif choice=="Svc":
  print("SVC:")
  model  = svm.SVC(C=2)
else:
    print("KNN")
    model=KNeighborsClassifier(n_neighbors=50) 


model.fit(X_train,y_train)


yt_p = model.predict(X_train)
yv_p = model.predict(X_val)

pt=precision_score(yt_p,y_train)
rt=recall_score(yt_p,y_train)
f1t=f1_score(yt_p,y_train)
mt=metrics.confusion_matrix(yt_p,y_train)


pv=precision_score(yv_p,y_val)
rv=recall_score(yv_p,y_val)
f1v=f1_score(yv_p,y_val)
mv=metrics.confusion_matrix(yv_p,y_val)

print('Training Accuracy', np.mean(yt_p==y_train),"\nTraining precision",pt,"\nTraining recall",rt,"\nTrianing f1 score",f1t)
print("Confusion Matrix for training period:",mt,"\n")

print('Validation  Accuracy', np.mean(yv_p==y_val),"\nValidation precision",pv,"\nValidation recall",rv,"\nValidation f1 score",f1v)
print("Confusion Matrix for validation period:",mv)

SVC:
Training Accuracy 0.7413793103448276 
Training precision 0.7562425683709869 
Training recall 0.7219069239500567 
Trianing f1 score 0.7386759581881533
Confusion Matrix for training period: [[654 205]
 [245 636]] 

Validation  Accuracy 0.6907630522088354 
Validation precision 0.7283582089552239 
Validation recall 0.6354166666666666 
Validation f1 score 0.678720445062587
Confusion Matrix for validation period: [[272  91]
 [140 244]]


# 9 Conclusions

After training different models and comparing their effeciency through the defined metrices SVM was found out to be the best.

Both the classes were equally likely correctly predicted hence the classifier is not bisaed towards any one of them.

The performace of the classifier was improved via normalizing the values of the input features for both training and validation dataset.

Also increasing the cost parameter from 1 till 2 gave more accuracy, hence with a little higher cost of misclassification a better result was acheived.

The scores of the best model i.e., SVM for predicting an audio as indoors or outdoors are as follows:

SVC:

Training Accuracy 0.7413793103448276 

Training precision 0.7562425683709869 

Training recall 0.7219069239500567 

Trianing f1 score 0.7386759581881533

Confusion Matrix for training period: 

[[654 205]

[245 636]] 

Validation  Accuracy 0.6907630522088354

Validation precision 0.7283582089552239 

Validation recall 0.6354166666666666 

Validation f1 score 0.678720445062587

Confusion Matrix for validation period: 

[[272  91]

[140 244]]
